Get Weather from API

In [1]:
import requests # request data through APIs
import pandas as pd
import time # for sleep time function

#### Adding API key in .env file
we need to define a variable that includes our API key that will be used for the appid parameter, because we need to authenticate ourselves to the API when we send a request, so the API knows that we have a registered account and can notify us in case we reach our query limit.

To do this,

1- go to user profile and go to 'My API key'

2- copy API key

3- add a variable called 'meteostat_api_key' to our .env file and put our API key in there, and save it!


In [2]:
# To be able to pull the API key from the .env, we need to import and run the load_dotenv function from the dotenv module first.
from dotenv import load_dotenv
load_dotenv()

True

#### API call
Next, as mentioned above the URL of the API has to be specified. In this example we are going to pull current weather data. The documentation for this API can be found here: https://rapidapi.com/meteostat/api/meteostat/

### URL Description
The url shown below is what needs to be used in order to connect to the API.

We can see that the url has

- a fixed part: 'https://meteostat.p.rapidapi.com/point/hourly?'
- and a variable part: 'lat:, lon:, start:, end:, rapidapi-key:, rapidapi_host:'

## Paramters Description
While the fixed part stays constant, the variable part consists of query strings or parameters, some optional some mandatory, which can be used to select or filter data.

Below are the optional and required parameters when pulling current weather data by latitude and longitude.

- latitude
- longitude
- start
- end
- rapidapi_key
- rapidapi_host

### Final API Call in loop for all airports
As the final step before we can do our first API call, we have to put all necessary parameters we chose (our API key, paramters) into a dictionary. We call it "parameters", but you can choose any name.

In [ ]:
# Set the url
url = 'https://meteostat.p.rapidapi.com/point/hourly?'

# Set locations
lat_and_lon = [['33.6367', '-84.428101'],
                ['33.56290054', '-86.75350189'],
                ['32.3111991882', '-90.0758972168'],
                ['34.729400634799994', '-92.2242965698'],
                ['29.99340057373047', '-90.25800323486328'],
                ['36.281898', '-94.306801']]

airport_codes = ['ATL', 'BHM', 'JAN', 'LIT', 'MSY', 'XNA']               

# Create empty dataframe, will be used to append each location's weather data
meteostat_details_df = pd.DataFrame([])

# Loop through all locations
for index, lonlat in enumerate(lat_and_lon):
    
    parameters = {
    'lat': lat_and_lon[index][0],
    'lon': lat_and_lon[index][1],
    'start': '2019-07-11',
    'end': '2019-07-15',
    'rapidapi-key': os.getenv('meteostat_api_key'),
    'rapidapi-host': 'meteostat.p.rapidapi.com'}

    print(parameters)
    # Create final url
    #url_f = url + lonlat + parameters
    
    # Request data from url
    r = requests.get(url, parameters)
    
    time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    meteostat_details_temp = r.json()

    # Flatten json response
    meteostat_details_temp_df = pd.json_normalize(meteostat_details_temp, sep="_", record_path='data')

    print(airport_codes[index])
    
    meteostat_details_temp_df['airport_code'] = airport_codes[index]
    
    # concatenate dataframes
    meteostat_details_df = pd.concat([meteostat_details_df, meteostat_details_temp_df], ignore_index=True)

# Print final dataset weather_df
meteostat_details_df

In [ ]:
meteostat_details_df[meteostat_details_df['airport_code']=='XNA']['coco'].value_counts()

In [ ]:
meteostat_weather_df = meteostat_details_df[meteostat_details_df['airport_code']!='XNA'].copy()

In [ ]:
meteostat_weather_df

### Inserting meteostat data into the database
The last step is to write this table into our database. We already created functions to help us do this using the sql_functions.py file. If the credentials and functions are set up correctly from the previous notebook, we can go ahead and import the helper function from the sql_functions.py file to get our connection engine.

#### Get Engine, Set table name and schema
Next, set the table name variable. This will be name of the table that will be written to the PostgreSQL database.

In [ ]:
from sql_functions import get_engine
schema = 'cgn_analytics_22_3' # UPDATE 'TABLE_SCHEMA' based on schema used in class 
engine = get_engine() # assign engine to be able to query against the database
table_name = 'meteostat_jsmn'

### Write the dataset to the database
The final step is to write the dataset to the database.

Complete the code below and write the dataset stored in planes_in_both to the PostgreSQL database.

In [ ]:
import psycopg2
if engine!=None:
    try:
        meteostat_weather_df.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None